### Non-linear detuning from sextupoles

Non-linear detuning can be produces by multipoles with even order in phase space coordinates (eg octupoles, sextupoles etc).

Sextupoles induce an amplitude dependent tune shift as defined by the first order hanarmonicities $\alpha_{xx}$, $\alpha_{xy}$, $\alpha_{yy}$ such that

\begin{equation}
\binom{\Delta \nu_x}{\Delta \nu_y} =\bigl(\begin{smallmatrix}
\alpha_{xx} & \alpha_{xy}\\ 
\alpha_{yx} & \alpha_{yy}
\end{smallmatrix}\bigr) \binom{2 J_x }{2 J_y}
\end{equation}

, where  $\alpha_{xy}$ =  $\alpha_{yx}$ - imposed by the conditions of symplecticity [2].

However, sextupoles generate these anharmonicities (coefficients) only in second order. They can be computed from the formulas (A.99-100-101) at page 165 of the following document:
https://cds.cern.ch/record/1644761/files/CERN-THESIS-2013-257.pdf


More details for the non-linear detuning from sextupoles can be found at the books listed  below:

[1] Accelerator Physics, S.Y.Lee p.198

[2] Beam Dynamics in High Energy particle accelerators, Andy Wolski p.389

### Prequesties
- os.popen('sh setup_environment.sh') # set up an enviroment where you can run mad-x
- access in /afs/ --> kinit and aklog

In [1]:
import os
import pandas as pd
import numpy as np
from math import *
from dotted_dict import DottedDict
from cpymad.madx import Madx

In [2]:
# MAD-X parameters dictionary
Qx = 26.130
Qy = 26.180
Qpx = 0.0
Qpy = 0.0
madx_settings = {'QH':Qx, 'QV':Qy, 'QPH':Qpx, 'QPV':Qpy}
use_aperture = True #False #
seq_name = 'sps'
harmonic_number = 4620

### Step 1: Identify lengths and strengths of different type of magnets

- This process is done manually by accessing the files sps.ele and elements.str.
- The values are stored in the data frame "df".

#### Parameters for the sextupole magnets.
From madx/sps/elements/sps.ele we see that ther are four types of sextupole magnets in the SPS lattice. LSD and LSF stand for the sextupole lenses used for chromatic correction (focusing and defocusing repsectively) Note also that for each type (SF or SD) there are two kidns indicated by A and B in the variables of their strengt (/madx/strength/elements.str). LSE and LSEN stand for the sextupole lenses used for the extraction (LSEN new magnet). In the nominal SPS, lattice only the chromatic sextupoles are on. This can be verified form the values of the strengh one can find in the sps_thin.seq.

In [3]:
strengths = DottedDict() # obtained from sps_thin.seq
strengths.klsda = -0.040896
strengths.klsdb = -0.063333733
strengths.klsfa = strengths.klsfc = 0.04516855
strengths.klsfb = 0.026760516

In [4]:
lengths = DottedDict() # obtained from madx/sps/elements/sps.ele
lengths.lsd = 0.42
lengths.lsf = 0.423

In [5]:
df = pd.DataFrame(columns=['name', 'type', 'strength', 'length'])

In [6]:
with open("./madx/sps/strength/elements.str", "r") as f:
    for x in f:
        if 'LSD' in x:
            if x[28:-2] == 'kLSDA':
                strength =  strengths.klsda
            else:
                strength =  strengths.klsdb
            df = df.append({'name':x[1:10], 'type':x[28:-2], 'strength':strength , 'length':lengths.lsd }, ignore_index=True)
        if 'LSF' in x:
            if x[28:-2] == 'kLSFA' or  x[28:-2] == 'kLSFC':
                strength =  strengths.klsfa
            else:
                strength =  strengths.klsfb
            df = df.append({'name':x[1:10], 'type':x[28:-2], 'strength':strength , 'length':lengths.lsf }, ignore_index=True)

### Step 2: Run MAD-X to create the lattice and extract the twiss table

In [7]:
mad = Madx()
mad.options.echo = False
mad.options.info = False
mad.warn = False
mad.chdir('madx');
mad.call('sps_thin_crabcavity.madx')
for parameter in madx_settings:
    setting = madx_settings[parameter]
    mad.input(f'{parameter} = {setting};')
mad.call('./sps/cmd/sps_matching.cmd')
mad.input('exec, SPS_matchtunes(QH, QV);')
mad.input('exec, SPS_setchroma_Q26(QPH, QPV);')
mad.input('acta.31637, harmon=%d;'%harmonic_number)


  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.05.01  (64 bit, Linux)       +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2019.06.07               +
  + Execution date: 2020.03.09 15:37:48      +
  ++++++++++++++++++++++++++++++++++++++++++++
dpp                =     0.001071393461 ;


return;

 call, file = 'sps/strength/elements.str';

  /**********************************************************************************

  *

  * SPS Ring version STUDY in MAD X SEQUENCE format

  * Generated the 25-MAR-2015 13:22:33 from LHCLAYOUT@EDMSDB Database

  *

  ***********************************************************************************/



/************************************************************************************/

/*                       STRENGTHS                                                  */

/************************************************************************************/



 MBA.10030          , ANGLE := kMBA;

 MBA.

True

In [8]:
# twiss
twtable = mad.twiss()
names = twtable.name # contains the name of every element in the lattice

enter Twiss module

++++++ table: summ

            length             orbit5               alfa            gammatr 
         6911.5038                 -0     0.001908372004        22.89119588 

                q1                dq1            betxmax              dxmax 
             26.13       -1.744591156        111.3952842        4.901338129 

             dxrms             xcomax             xcorms                 q2 
       2.348345288                  0                  0              26.18 

               dq2            betymax              dymax              dyrms 
     -0.8536795346        108.6334011                  0                  0 

            ycomax             ycorms             deltap            synch_1 
                 0                  0                  0                  0 

           synch_2            synch_3            synch_4            synch_5 
                 0                  0                  0                  0 

            nflips 
          

### Step 3: Save in a data frame the information of the lattice

In [9]:
lattice_DF=mad.table.twiss.dframe()

In [10]:
lattice_DF

name      keyword          s       betx      alfx  \
mystart              mystart:1       marker     0.0000  29.238974 -0.875765   
cravity.1          cravity.1:1  rfmultipole     0.0000  29.238974 -0.875765   
drift_0[0]           drift_0:0        drift     0.6000  30.311648 -0.912024   
cravity.2          cravity.2:1  rfmultipole     0.6000  30.311648 -0.912024   
drift_1[0]           drift_1:0        drift    10.2039  53.403549 -1.492405   
...                        ...          ...        ...        ...       ...   
bpce.61705        bpce.61705:1      monitor  6901.3093  21.970650  0.581066   
drift_3250[0]     drift_3250:0        drift  6903.8725  19.391874  0.425011   
qda.61710          qda.61710:1    multipole  6903.8725  19.391874 -0.414592   
drift_3251[0]     drift_3251:0        drift  6911.5038  29.238974 -0.875765   
spsmystart_p_  spsmystart_p_:1       marker  6911.5038  29.238974 -0.875765   

                     mux        bety      alfy        muy    x  ...  sig54  \
mystart         0.000000   76.073157  1.898525   0.000000  0.0  ...    0.0   
cravity.1       0.000000   76.073157  1.898525   0.000000  0.0  ...    0.0   
drift_0[0]      0.003208   73.816716  1.862210   0.001274  0.0  ...    0.0   
cravity.2       0.003208   73.816716  1.862210   0.001274  0.0  ...    0.0   
drift_1[0]      0.041569   43.630369  1.280925   0.028338  0.0  ...    0.0   
...                  ...         ...       ...        ...  ...  ...    ...   
bpce.61705     26.058262   96.968126 -2.187560  26.162644  0.0  ...    0.0   
drift_3250[0]  26.078077  108.574419 -2.340488  26.166620  0.0  ...    0.0   
qda.61710      26.078077  108.574419  2.360417  26.166620  0.0  ...    0.0   
drift_3251[0]  26.130000   76.073157  1.898525  26.180000  0.0  ...    0.0   
spsmystart_p_  26.130000   76.073157  1.898525  26.180000  0.0  ...    0.0   

                      sig55  sig56  sig61  sig62  sig63  sig64  sig65  sig66  \
mystart        0.000000e+00    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
cravity.1      0.000000e+00    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
drift_0[0]     0.000000e+00    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
cravity.2      0.000000e+00    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
drift_1[0]     0.000000e+00    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
...                     ...    ...    ...    ...    ...    ...    ...    ...   
bpce.61705     2.426979e-09    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
drift_3250[0]  2.426979e-09    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
qda.61710      2.426979e-09    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
drift_3251[0]  2.426979e-09    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
spsmystart_p_  2.426979e-09    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

                n1  
mystart        0.0  
cravity.1      0.0  
drift_0[0]     0.0  
cravity.2      0.0  
drift_1[0]     0.0  
...            ...  
bpce.61705     0.0  
drift_3250[0]  0.0  
qda.61710      0.0  
drift_3251[0]  0.0  
spsmystart_p_  0.0  

[6506 rows x 256 columns]

### Step 4: Crate the data frame sextupoles_DF which will contain information for every sextupole magnet in the lattice which is necessary for the calculation of the coefficients.

- Iterate through the names of all the elements in the lattice
- Found the oned that match with the name of the elements in the "df"
- Save their properties in "sextupoles_DF"

In [16]:
# open the twiss, iterate throught the elements, if it mathced with any of the ones in the data frame, then save the beta function and the phase advance
# extract from the lattice DF a temporary one which includes only the info (betx mux, bety, muy) for the sextupoles
sextupoles_DF = pd.DataFrame(columns=['name', 'type', 'strength', 'length', 'betax', 'mux', 'betay', 'muy'])

for index_1, name_1 in enumerate(names):
    for index_2, name_2 in enumerate(df['name']):
        if name_1[:-2] == name_2.lower(): # string.lower() make the string case insensitive
            sextupoles_DF = sextupoles_DF.append({'name':df['name'][index_2], 'type':df['type'][index_2], 'strength':df['strength'][index_2], 'length':df['length'][index_2], 'betax':lattice_DF['betx'][index_1], 'mux':lattice_DF['mux'][index_1], 'betay':lattice_DF['bety'][index_1], 'muy':lattice_DF['muy'][index_1]}, ignore_index=True)

In [17]:
sextupoles_DF

name   type  strength  length      betax        mux      betay  \
0    LSF.62005  kLSFA  0.045169   0.423  97.647420   0.308688  22.428902   
1    LSD.62305  kLSDB -0.063334   0.420  22.462653   0.657132  96.719197   
2    LSF.62405  kLSFA  0.045169   0.423  97.655096   0.792555  22.419784   
3    LSD.62505  kLSDA -0.040896   0.420  22.563851   0.897580  96.625972   
4    LSF.62605  kLSFB  0.026761   0.423  95.912299   1.034262  22.470992   
..         ...    ...       ...     ...        ...        ...        ...   
105  LSF.61205  kLSFA  0.045169   0.423  96.972085  25.467787  22.421499   
106  LSF.61205  kLSFC  0.045169   0.423  96.972085  25.467787  22.421499   
107  LSD.61305  kLSDA -0.040896   0.420  21.891687  25.575148  97.057814   
108  LSF.61405  kLSFB  0.026761   0.423  96.374499  25.714211  22.479359   
109  LSD.61505  kLSDB -0.063334   0.420  23.108437  25.818590  96.354035   

           muy  
0     0.333822  
1     0.712355  
2     0.818344  
3     0.954884  
4     1.060788  
..         ...  
105  25.541967  
106  25.541967  
107  25.678091  
108  25.783694  
109  25.920287  

[110 rows x 8 columns]

In [18]:
# two sextupoles are defined two times kLSFA, kLSFC  (it's a bag to be fixed in the next verison of MADX)
# type A and type C are the same today. Type C is a different family that had different settings when SPS was used 
# as a collider. Remove them.. The total number of sextupoles is 108.

In [19]:
counter = 0
for i in range(0, len(sextupoles_DF['name'])-1):
    if sextupoles_DF['name'][i] == sextupoles_DF['name'][i+1]:
        sextupoles_DF = sextupoles_DF.drop(i) # drop row by index
        print('Row with index {} dropped'.format(i))
sextupoles_DF = sextupoles_DF.reset_index()
print('The new length of axis 0 of the data frame is {}'.format(len(sextupoles_DF['name'])))

Row with index 102 dropped
Row with index 105 dropped
The new length of axis 0 of the data frame is 108


In [20]:
sextupoles_DF

index       name   type  strength  length      betax        mux  \
0        0  LSF.62005  kLSFA  0.045169   0.423  97.647420   0.308688   
1        1  LSD.62305  kLSDB -0.063334   0.420  22.462653   0.657132   
2        2  LSF.62405  kLSFA  0.045169   0.423  97.655096   0.792555   
3        3  LSD.62505  kLSDA -0.040896   0.420  22.563851   0.897580   
4        4  LSF.62605  kLSFB  0.026761   0.423  95.912299   1.034262   
..     ...        ...    ...       ...     ...        ...        ...   
103    104  LSD.61105  kLSDB -0.063334   0.420  23.149994  25.334413   
104    106  LSF.61205  kLSFC  0.045169   0.423  96.972085  25.467787   
105    107  LSD.61305  kLSDA -0.040896   0.420  21.891687  25.575148   
106    108  LSF.61405  kLSFB  0.026761   0.423  96.374499  25.714211   
107    109  LSD.61505  kLSDB -0.063334   0.420  23.108437  25.818590   

         betay        muy  
0    22.428902   0.333822  
1    96.719197   0.712355  
2    22.419784   0.818344  
3    96.625972   0.954884  
4    22.470992   1.060788  
..         ...        ...  
103  96.288649  25.435706  
104  22.421499  25.541967  
105  97.057814  25.678091  
106  22.479359  25.783694  
107  96.354035  25.920287  

[108 rows x 9 columns]

### Step 5: Compute the detuning coefficients

\begin{equation}
\alpha_{xx} = \frac{\partial \Delta \nu_x}{\partial J_x}
\end{equation}

\begin{equation}
\alpha_{xy} = \frac{\partial \Delta \nu_x}{\partial J_y} = \frac{\partial \Delta \nu_y}{\partial J_x}
\end{equation}

\begin{equation}
\alpha_{yy} = \frac{\partial \Delta \nu_y}{\partial J_y}
\end{equation}


Note: The unit of the coefficients is $(\pi m)^{-1}$

In [21]:
n = len(sextupoles_DF['name'])
my_sum = 0


for i in range(0, n):
    for k in range(0, n):
        k2L_f = sextupoles_DF['strength'][i]*sextupoles_DF['length'][i]*sextupoles_DF['strength'][k]*sextupoles_DF['length'][k]
        betax_f = (sextupoles_DF['betax'][i]**(3/2.))*(sextupoles_DF['betax'][k]**(3/2.)) 
        frac1 = (3*np.cos(abs(sextupoles_DF['mux'][k]- sextupoles_DF['mux'][i])-np.pi*Qx))/np.sin(np.pi*Qx)
        frac2 = (np.cos(abs(sextupoles_DF['mux'][k]- sextupoles_DF['mux'][i])-3*np.pi*Qx))/np.sin(3*np.pi*Qx)
        
        my_sum+= k2L_f*betax_f*(frac1+frac2)
axx = my_sum/(-64*np.pi)
print('axx = {}'.format(axx))

axx = -733.448679521935


In [22]:
n = len(sextupoles_DF['name'])
my_sum = 0


for i in range(0, n):
    for k in range(0, n):
        k2L_f = sextupoles_DF['strength'][i]*sextupoles_DF['length'][i]*sextupoles_DF['strength'][k]*sextupoles_DF['length'][k]
        betax_f = np.sqrt(sextupoles_DF['betax'][i]*sextupoles_DF['betax'][k])*sextupoles_DF['betay'][i]
        frac1 = (2*sextupoles_DF['betax'][k]*np.cos(abs(sextupoles_DF['mux'][k]-sextupoles_DF['mux'][i])-np.pi*Qx))/(np.sin(np.pi*Qx))
        frac2 = (sextupoles_DF['betay'][k]*np.cos(abs(sextupoles_DF['mux'][k]-sextupoles_DF['mux'][i]+2*(sextupoles_DF['muy'][k]-sextupoles_DF['muy'][i]))-np.pi*(Qx+2*Qy))/(np.sin(np.pi*(Qx+2*Qy))))
        frac3 = (sextupoles_DF['betay'][k]*np.cos(abs(sextupoles_DF['mux'][k]-sextupoles_DF['mux'][i]-2*(sextupoles_DF['muy'][k]-sextupoles_DF['muy'][i]))-np.pi*(Qx-2*Qy))/(np.sin(np.pi*(Qx-2*Qy))))

        my_sum+=k2L_f*betax_f*(frac1-frac2+frac3)
axy = my_sum/(32*np.pi)
print('axy = {}'.format(axy))

axy = -308.8714665421183


In [23]:
n = len(sextupoles_DF['name'])
my_sum = 0


for i in range(0, n):
    for k in range(0, n):
        k2L_f = sextupoles_DF['strength'][i]*sextupoles_DF['length'][i]*sextupoles_DF['strength'][k]*sextupoles_DF['length'][k]
        betax_f = np.sqrt(sextupoles_DF['betax'][i]*sextupoles_DF['betax'][k])*sextupoles_DF['betay'][i]*sextupoles_DF['betay'][k]
        frac1 = (4*np.cos(abs(sextupoles_DF['mux'][k]- sextupoles_DF['mux'][i]))-np.pi*Qx)/(np.sin(np.pi*Qx))
        frac2 = (np.cos(abs(sextupoles_DF['mux'][k]-sextupoles_DF['mux'][i]+2*(sextupoles_DF['muy'][k]-sextupoles_DF['muy'][i]))-np.pi*(Qx+2*Qy))/(np.sin(np.pi*(Qx+2*Qy))))
        frac3 = (np.cos(abs(sextupoles_DF['mux'][k]-sextupoles_DF['mux'][i]-2*(sextupoles_DF['muy'][k]-sextupoles_DF['muy'][i]))-np.pi*(Qx-2*Qy))/(np.sin(np.pi*(Qx-2*Qy))))
        
        my_sum+=k2L_f*betax_f*(frac1+frac2+frac3)
                  
ayy = my_sum/(-64*np.pi)
print('ayy = {}'.format(ayy))

ayy = 151909.02992948194
